# Radiative Processes for Black Holes

## 3. Synchrotron Radiation

This repository contains python functions that are useful for studying synchrotron radiation.

In [1]:
# Turn on autoreload for Jupyter to make developer easier
%load_ext autoreload
%autoreload 2

# Standard library and constants
from math import pi

# Standard python packages
import numpy as np
from matplotlib import pyplot as plt

# Astropy units and constants are extremely powerful
from astropy import units     as u
from astropy import constants as c

Although object oriented programming (OOP) is sometimes overkill for numerical analysis, given that we will need to compare different formulations of synchrotron radiation, it may still be useful to create classes to group functions/methods together.

In OOP, it is possible to declare the required methods in a class without defining them.  Such a class is called an abstract class.  We will use python's builtin Abstract Base Classes package for this.

Since the Planck law
$$
B_\nu = \frac{2 h \nu^3}{c^2} \frac{1}{\exp(h\nu/k_\mathrm{B}T) - 1}
$$
is simple enough, we include it as a concrete method.  Nevertheless, it is possible to override it with better optimized form.

In [2]:
from abc import ABC, abstractmethod

class Radiating(ABC):
    """ Radiating Abstract Base Class """
    
    def __init__(self, rho, T, cos_theta, B):
        """ Set fluid property """
        self.rho = rho
        self.T   = T
        self.B   = B
        self.cos = cos_theta
    
    def Bnu(self, nu):
        """ Planck function """
        x = (c.h * nu) / (c.k_B * self.T)
        f = 2 * c.h * (nu * nu * nu) / (c.c * c.c)
        return f / (np.exp(x) - 1)

    @abstractmethod
    def jnu_bremss(self, nu):
        """ Bremsstrahlung emissivity """
        pass
    
    @abstractmethod
    def jnu_sync(self, nu):
        """ Synchrotron emissivity """
        pass